# Docking our possible compounds with autodock Vina



Script to run docking on all 2500 possible compounds in the game. Based off of a tutorial on autodock vina (https://colab.research.google.com/github/pb3lab/ibm3202/blob/master/tutorials/lab06_docking.ipynb) 

Requires uploading of r_decomp.csv to the runtime in order to load the SMILES strings of all possible compounds.

IMPORTANT: At the moment, it only works with python 3.8. Colab updated (march 22) to python 3.9, so we are using the previous runtime. To do this, once you connnect to runtime, go to tools > command palette > use fallback runtime version. NB: This might only be available for the rest of march so a more sustainable solution might have to be found.

In [ ]:
#Installing py3Dmol using pip
!pip install py3Dmol
#Installing biopython using pip
!pip install biopython
#Installing pdb2pqr v3.0 using pip
!pip install pdb2pqr
#We will also install kora for using RDkit
!pip install kora
!pip install rdkit
#Importing py3Dmol for safety
import py3Dmol
#Checking that pdb2pqr was properly installed
!pdb2pqr30 -h
#Install conda using the new conda-colab library
!pip install -q condacolab==0.1.5

# import docking 
!wget https://vina.scripps.edu/wp-content/uploads/sites/55/2020/12/autodock_vina_1_1_2_linux_x86.tgz
!tar xzvf autodock_vina_1_1_2_linux_x86.tgz

import condacolab
condacolab.install_miniconda()



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
usage: pdb2pqr
       [-h]
       [--ff {AMBER,CHARMM,PARSE,TYL06,PEOEPB,SWANSON}]
       [--userff USERFF]
       [--clean]
       [--nodebump]
       [--noopt]
       [--keep-chain]
       [--assign-only]
       [--ffout {AMBER,CHARMM,PARSE,TYL06,PEOEPB,SWANSON}]
       [--usernames USERNAMES]
       [--apbs-input APBS_INPUT]
       [--pdb-output PDB_OUTPUT]
       [--ligand LIGAND]
       [--whitespace]
       [--neutraln]
       [--neutralc]
       [--drop-water]
       [--include-header]
       

In [ ]:
#Install MGLtools and OpenBabel from
#the bioconda repository
!conda install -c conda-forge -c bioconda mgltools openbabel zlib --yes

/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 23.1.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - mgltools
    - openbabel
    - zlib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.12.7  |       ha878542_0         143 KB  conda-forge
    cairo-1.16.0               |    h18b612c_1001         1.5 MB  conda-forge
    certifi-2022.12.7          |     pyhd8ed1ab_0         147 KB  conda-forge
    conda-4.14.0               |   py38h578d9bd_0        1012 KB  conda-forge
    fon

In [ ]:
#Download and extract Autodock Vina from SCRIPPS
#Then, we set up an alias for vina to be treated as a native binary
%alias vina /content/autodock_vina_1_1_2_linux_x86/bin/vina
%alias vina_split /content/autodock_vina_1_1_2_linux_x86/bin/vina_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Downloading the PDB files using biopython
import os
from Bio.PDB import *
# MMP-12 crystal structure
pdbid = ['3ehy']
pdbl = PDBList()
for s in pdbid:
  pdbl.retrieve_pdb_file(s, pdir='.', file_format ="pdb", overwrite=True)
  os.rename("pdb"+s+".ent", s+".pdb")

In [ ]:
#This script will create a folder called "single-docking" for our experiment
#Then, it will print all "ATOM" and "TER" lines from a given PDB into a new file

#Let's make a folder first. We need to import the os and path library
import os
from pathlib import Path 

#Then, we define the path of the folder we want to create.
#Notice that the HOME folder for a hosted runtime in colab is /content/
singlepath = Path("/content/drive/MyDrive/single-dock/")

#Now, we create the folder using the os.mkdir() command
#The if conditional is just to check whether the folder already exists
#In which case, python returns an error
if os.path.exists(singlepath):
  print("path already exists")
if not os.path.exists(singlepath):
  os.mkdir(singlepath)
  print("path was succesfully created")

#Now we assign a variable "protein" with the name and extension of our pdb
# changed to reflect MMP-12 entry
protein = "3ehy.pdb"

#And we use the following script to selectively print the lines that contain the
#string "ATOM" and "TER" into a new file inside our recently created folder
with open(singlepath / "3ehy_prot.pdb","w") as g:
  f = open(protein,'r')
  for line in f:
    row = line.split()
    if row[0] == "ATOM":
      g.write(line)
    elif row[0] == "TER":
      g.write("TER\n")
  g.write("END")
  print("file successfully created")

path was succesfully created
file successfully created


In [ ]:
#Parameterizing and adding Gasteiger charges into our protein using MGLtools
!prepare_receptor4.py -r $singlepath/3ehy_prot.pdb -o $singlepath/3ehy_prot.pdbqt -A hydrogens -U nphs_lps -v

/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
set verbose to  True
read  /content/drive/MyDrive/single-dock-drive2/3ehy_prot.pdb
WARNING! 3ehy_prot has 5  alternate location atoms!
Use prepare_pdb_split_alt_confs.py to create pdb files containing a single conformation.

setting up RPO with mode= automatic and outputfilename=  /content/drive/MyDrive/single-dock-drive2/3ehy_prot.pdbqt
charges_to_add= gasteiger
delete_single_nonstd_residues= None
adding gasteiger charges to peptide


In [ ]:
#Let's make a folder first. We need to import the os and path library
import os
from pathlib import Path

#We will first create a path for all ligands that we will use in this tutorial
#Notice that the HOME folder for a hosted runtime in colab is /content/
ligandpath = Path("/content/ligands/")

#Now, we create the folder using the os.mkdir() command
#The if conditional is just to check whether the folder already exists
#In which case, python returns an error
if os.path.exists(ligandpath):
  print("ligand path already exists")
if not os.path.exists(ligandpath):
  os.mkdir(ligandpath)
  print("ligand path was succesfully created")

ligand path was succesfully created


PLEASE 
1. upload r_group_decomp.csv to the /content/ directory before running the following **cells** 

In [ ]:
os.chdir('/content')

In [ ]:
from csv import reader
with open('r_group_decomp.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    header = next(csv_reader)
    # Check file as empty
    if header != None:
        # Iterate over each row after the header in the csv
        for i,row in enumerate(csv_reader):
          # row variable is a list that represents a row in csv
          ligand = row[0]
          A_tag = row[2]
          B_tag = row[4]
          with open('ligands/{}.smiles'.format(A_tag + B_tag), 'w') as f:
            f.write("{}".format(ligand))
          

In [ ]:
import os
with open(singlepath / "config_singledock","w") as f:
  f.write("#CONFIGURATION FILE (options not used are commented) \n")
  f.write("\n")
  f.write("#INPUT OPTIONS \n")
  f.write("receptor = 3ehy_prot.pdbqt \n")
  f.write("ligand = ligand.pdbqt \n")
  f.write("#flex = [flexible residues in receptor in pdbqt format] \n")
  f.write("#SEARCH SPACE CONFIGURATIONS \n")
  f.write("#Center of the box (values bxi, byi and bzi) \n")
#CHANGE THE FOLLOWING DATA WITH YOUR BOX CENTER COORDINATES  
  f.write("center_x = -6 \n")
  f.write("center_y = 0 \n")
  f.write("center_z = -12 \n")
#CHANGE THE FOLLOWING DATA WITH YOUR BOX DIMENSIONS
  f.write("#Size of the box (values bxf, byf and bzf) \n")
  f.write("size_x = 27 \n")
  f.write("size_y = 29 \n")
  f.write("size_z = 29 \n")
  f.write("#OUTPUT OPTIONS \n")
  f.write("#out = \n")
  f.write("#log = \n")
  f.write("\n")
  f.write("#OTHER OPTIONS \n")
  f.write("#cpu =  \n")
  f.write("exhaustiveness = 8 \n")
  f.write("num_modes = 1\n")
  f.write("#energy_range = \n")
  f.write("#seed = ")


In [ ]:
import py3Dmol
import kora.install.rdkit
!conda install -c conda-forge rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
protein = "3ehy.pdb"

# Use os.listdir to get a list of all the text files in the directory
text_files = os.listdir(ligandpath)

text_files.sort()

# Iterate through the list of text files
for i,text_file in enumerate(text_files):
    mol = text_files[i][0:6]

    # Construct the full path to the text file
    file_path = os.path.join(ligandpath, text_file)
    # Run the obabel command for the text file
    !obabel $file_path -O ligand.mol2 --gen3d best -p 7.4 --canonical
    !prepare_ligand4.py -l ligand.mol2 -o $singlepath/ligand.pdbqt -U nphs_lps -v
    os.remove("ligand.mol2")
    #Changing directory to the single docking folder
    os.chdir(singlepath)
    #Executing AutoDock Vina with our configuration file
    %vina --config config_singledock --out output.pdbqt --log log.txt
    os.rename('log.txt','log{}.txt'.format(mol))
    #Exiting the execution directory

    # saves docked compound + log.txt with affinity to drive
    dock_path = Path(singlepath,mol + "dock.pdb")
    !obabel -ipdbqt $singlepath/output.pdbqt -opdb -O $dock_path -m



Streaming output truncated to the last 5000 lines.

Detected 4 CPUs
Reading input ... done.
Setting up the scoring function ... done.
Analyzing the binding site ... done.
Using random seed: 1984133397
Performing search ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************
done.
Refining results ... done.

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1         -9.0      0.000      0.000
Writing output ... done.
/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
1 molecule converted
/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
1 molecule converted
/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
set verbose to  True
read  ligand.mol2
setting up LPO with m

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-c8b226dbaf24>", line 31, in <module>
    os.rename('log.txt','log{}.txt'.format(mol))
OSError: [Errno 107] Transport endpoint is not connected: 'log.txt' -> 'logA44B41.txt'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  F

OSError: ignored

In [ ]:
# SAVES CSV FILE WITH AFFINITY SCORE EXTRACTED FROM LOG.TXT FILES

directory = single_path

# create a list to store the results
results = []
# loop through all the log files in the directory
files = os.listdir(directory)
files.sort()
for filename in files:
    if filename.endswith('.txt'):
        # extract the labels from the filename
        label = filename[3:9]
        # read the file and extract the affinity score
        with open(os.path.join(directory, filename), 'r') as f:
            for line in f:
                if line.startswith('   1'):
                    affinity = line.split()[1]
                    break
        # add the label and affinity to the results list
        results.append([label, affinity])
# write the results to a CSV file
with open('affinities.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Labels', 'Affinity (kcal/mol)'])
    writer.writerows(results) 